In [16]:
from langchain_community.utilities import SQLDatabase
import os
import pandas as pd
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from sqlalchemy import create_engine


In [6]:
%pwd

'c:\\Users\\adminn\\Documents\\GitHub\\AI-Database-Agent\\Notebooks'

In [7]:
os.chdir('..//')

In [20]:
db_path="data"+"/sqldb.db"
db=SQLDatabase.from_uri(f"sqlite:///{db_path}")
print(db.get_usable_table_names)

db.run(
    " SELECT * FROM Employee LIMIT 10;"
)

<bound method SQLDatabase.get_usable_table_names of <langchain_community.utilities.sql_database.SQLDatabase object at 0x000001EE93DDE8B0>>


"[(1, 'Adams', 'Andrew', 'General Manager', None, '1962-02-18 00:00:00', '2002-08-14 00:00:00', '11120 Jasper Ave NW', 'Edmonton', 'AB', 'Canada', 'T5K 2N1', '+1 (780) 428-9482', '+1 (780) 428-3457', 'andrew@chinookcorp.com'), (2, 'Edwards', 'Nancy', 'Sales Manager', 1, '1958-12-08 00:00:00', '2002-05-01 00:00:00', '825 8 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 2T3', '+1 (403) 262-3443', '+1 (403) 262-3322', 'nancy@chinookcorp.com'), (3, 'Peacock', 'Jane', 'Sales Support Agent', 2, '1973-08-29 00:00:00', '2002-04-01 00:00:00', '1111 6 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 5M5', '+1 (403) 262-3443', '+1 (403) 262-6712', 'jane@chinookcorp.com'), (4, 'Park', 'Margaret', 'Sales Support Agent', 2, '1947-09-19 00:00:00', '2003-05-03 00:00:00', '683 10 Street SW', 'Calgary', 'AB', 'Canada', 'T2P 5G3', '+1 (403) 263-4423', '+1 (403) 263-4289', 'margaret@chinookcorp.com'), (5, 'Johnson', 'Steve', 'Sales Support Agent', 2, '1965-03-03 00:00:00', '2003-10-17 00:00:00', '7727B 41 Ave', 'Calgar

# Q&A DataBase

In [12]:
import warnings
import openai
warnings.filterwarnings('ignore')
from dotenv import load_dotenv


In [26]:
from openai import OpenAI
from langchain.chat_models import ChatOpenAI
import os

open_api_key=os.getenv('OPENAI_API_KEY')

llm=ChatOpenAI(
    model='gpt-4',
    temperature=0,
    openai_api_key=open_api_key
)


In [17]:
llm.invoke('what is meaning of life?')

AIMessage(content="The meaning of life is a philosophical and metaphysical question related to the purpose or significance of life or existence in general. This concept can be interpreted in many different ways depending on one's beliefs. Some people believe the meaning of life is to pursue happiness or personal fulfillment, while others believe it is to contribute to the betterment of humanity, seek knowledge, or fulfill certain duties or responsibilities. In religious contexts, the meaning of life is often believed to be a divine purpose or plan. Ultimately, the answer can be deeply personal and subjective.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 109, 'prompt_tokens': 13, 'total_tokens': 122, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4', 'system_fingerprint': None,

In [24]:
from langchain.chains import create_sql_query_chain
chain=create_sql_query_chain(llm,db)
response=chain.invoke({'question':'how many artistis are there?'})
response

'SELECT COUNT("ArtistId") FROM "Artist"'

In [27]:
from langchain.chains import create_sql_query_chain
chain=create_sql_query_chain(llm,db)
response=chain.invoke({'question':'give me all column names that i have in  employee table?'})
response

'PRAGMA table_info(Employee);'

In [25]:
db.run(response)

'[(275,)]'

In [28]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

write_query=create_sql_query_chain(llm,db)
excute_query=QuerySQLDataBaseTool(db=db)
chain=write_query | excute_query

chain.invoke({"question":'how many employees are there'})

'[(8,)]'

In [29]:


prompt=PromptTemplate.from_template(

    """ Given the following user question,corresponding SQL query and SQL result and ansswer the user question
    
    Question: {question}
    SQL Query: {query}
    SQL Result: {result}
    Answer:"""
)

answer=prompt|llm|StrOutputParser()
chain=(
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | excute_query
    ) | answer
)

chain.invoke({"question":'give me the columns in employee table'})


'The columns in the employee table are EmployeeId, LastName, FirstName, Title, ReportsTo, BirthDate, HireDate, Address, City, State, Country, PostalCode, Phone, Fax, and Email.'

In [14]:
import yaml
def read_yaml_file(path:str):

    with open(path) as f:

        app_config=yaml.load(f,Loader=yaml.FullLoader)

    return app_config

In [16]:
app_config=read_yaml_file('configs/config.yaml')
app_config

{'directories': {'stored_csv_xlsx_directory': 'data/csv_data',
  'sqldb_directory': 'data/sqldb.db',
  'uploaded_files_sqldb_directory': 'data/uploaded_files_sqldb.db',
  'stored_csv_xlsx_sqldb_directory': 'data/csv_xlsx_sqldb.db',
  'persist_directory': 'data/chroma'},
 'llm_config': {'agent_llm_system_role': 'Given the following user question, corresponding SQL query, and SQL result, answer the user question.\n Question: {question}\n SQL Query: {query}\n SQL Result: {result}\n Answer: ',
  'rag_llm_system_role': "You will recieve the user's question along with the search results of that question over a database. Give the user the proper answer.",
  'engine': 'gpt-35-turbo',
  'temperature': 0},
 'rag_config': {'collection_name': 'titanic', 'top_k': 1}}

In [17]:
from sqlalchemy import create_engine,inspect

In [42]:
db_path=app_config['directories']['stored_csv_xlsx_sqldb_directory']
db_path=f"sqlite:///{db_path}"
engine=create_engine(db_path)
engine


Engine(sqlite:///data/csv_xlsx_sqldb.db)

In [44]:
csv_data=app_config['directories']['stored_csv_xlsx_directory']
files_list=os.listdir(csv_data)
files_list

['cancer.csv', 'diabetes.csv']

In [ ]:

for file in files_list:
    full_file_path=os.path.join(csv_data,file)
    file_name,file_extension=os.path.splitext(file)
    if file_extension == '.csv':

        df=pd.read_csv(full_file_path)
    elif file_extension == '.xlsx':

        df=pd.read_excel(full_file_path)

    else:
        raise NotImplementedError('The expected extension is not implemented')
    
    df.to_sql(file_name,engine,index=False,if_exists='replace')

    


In [46]:
inspection=inspect(engine)
table_names=inspection.get_table_names()
table_names

['cancer', 'diabetes']

# Q&A titanic dataset with agent

In [28]:
df=pd.read_csv('data/uploaded_data/titanic_dataset.csv')
db_path="data"+"test_sqldb.db"
db_path=f"sqlite:///{db_path}"
engine=create_engine(db_path)

df.to_sql("titanic",engine,index=False,if_exists='replace')

891

In [29]:
from langchain_community.agent_toolkits import create_sql_agent
db = SQLDatabase(engine=engine)
agent_excutor=create_sql_agent(
    llm,db=db,agent_type='openai-tools'
,verbose=True)

In [31]:
agent_excutor.invoke({"input":'what is the average age of survivors ?'})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


titanic
Invoking: `sql_db_schema` with `{'table_names': 'titanic'}`



CREATE TABLE titanic (
	"PassengerId" BIGINT, 
	"Survived" BIGINT, 
	"Pclass" BIGINT, 
	"Name" TEXT, 
	"Sex" TEXT, 
	"Age" FLOAT, 
	"SibSp" BIGINT, 
	"Parch" BIGINT, 
	"Ticket" TEXT, 
	"Fare" FLOAT, 
	"Cabin" TEXT, 
	"Embarked" TEXT
)

/*
3 rows from titanic table:
PassengerId	Survived	Pclass	Name	Sex	Age	SibSp	Parch	Ticket	Fare	Cabin	Embarked
1	0	3	Braund, Mr. Owen Harris	male	22.0	1	0	A/5 21171	7.25	None	S
2	1	1	Cumings, Mrs. John Bradley (Florence Briggs Thayer)	female	38.0	1	0	PC 17599	71.2833	C85	C
3	1	3	Heikkinen, Miss. Laina	female	26.0	0	0	STON/O2. 3101282	7.925	None	S
*/
Invoking: `sql_db_query_checker` with `{'query': 'SELECT AVG(Age) as average_age FROM titanic WHERE Survived = 1'}`
responded: The table 'titanic' contains the information we need. The 'Survived' column indicates whether a passenger survived (1) or not 

{'input': 'what is the average age of survivors ?',
 'output': 'The average age of survivors is approximately 28.34 years.'}

In [33]:
agent_excutor.invoke({"input":'what is the median age of survivors ?'})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


titanic
Invoking: `sql_db_schema` with `{'table_names': 'titanic'}`



CREATE TABLE titanic (
	"PassengerId" BIGINT, 
	"Survived" BIGINT, 
	"Pclass" BIGINT, 
	"Name" TEXT, 
	"Sex" TEXT, 
	"Age" FLOAT, 
	"SibSp" BIGINT, 
	"Parch" BIGINT, 
	"Ticket" TEXT, 
	"Fare" FLOAT, 
	"Cabin" TEXT, 
	"Embarked" TEXT
)

/*
3 rows from titanic table:
PassengerId	Survived	Pclass	Name	Sex	Age	SibSp	Parch	Ticket	Fare	Cabin	Embarked
1	0	3	Braund, Mr. Owen Harris	male	22.0	1	0	A/5 21171	7.25	None	S
2	1	1	Cumings, Mrs. John Bradley (Florence Briggs Thayer)	female	38.0	1	0	PC 17599	71.2833	C85	C
3	1	3	Heikkinen, Miss. Laina	female	26.0	0	0	STON/O2. 3101282	7.925	None	S
*/
Invoking: `sql_db_query` with `{'query': 'SELECT COUNT(*) FROM titanic WHERE Survived = 1'}`
responded: The table "titanic" contains the information we need. The "Survived" column indicates whether a passenger survived (1) or not (0), and the "Age" colu

{'input': 'what is the median age of survivors ?',
 'output': 'The median age of survivors is 25.0 years.'}

In [42]:
df[df['Survived']==1]['Age'].median()

28.0